# 20/03/2024
Resnet training for satellite images

In [1]:
from keras.src.applications.resnet_v2 import ResNet50V2
%load_ext autoreload
%autoreload 2

In [2]:
import random

import geopandas as gpd
import numpy as np
import seaborn
import tensorflow as tf
from keras.src.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt
from keras import Input, Model
from keras.src.losses import MeanSquaredError
from keras.src.optimizers import Adam

In [3]:
print('TensorFlow version: {}'.format(tf.__version__))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found - On for CPU time!')
else:
    print('Found GPU at {}'.format(device_name))

In [4]:
from tensorflow.python.platform import build_info as tf_build_info

tf_build_info.build_info

In [5]:
!/usr/local/cuda/bin/nvcc -V

In [6]:
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

In [7]:
# Check libcudnn8 version
!apt-cache policy libcudnn8

In [8]:
# set random seed / make reproducible
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [9]:
#load labels to images and observe it
eg_labels = gpd.read_file('../outputs/matched/gauteng-qol-cluster-tiles.geojson')
eg_labels = eg_labels[["tile", "qol_index"]]
print(len(eg_labels))
eg_labels.head()

In [10]:
plt.figure(figsize=(10, 6))
seaborn.set_style('whitegrid')
seaborn.set_palette('inferno')
seaborn.displot(eg_labels['qol_index'], rug=True, kde_kws={'fill':True, 'color':'r'})
plt.xlabel('QoL Index', fontsize=12, color='b')
plt.ylabel('Frequency', fontsize=12, color='b')
plt.title('Distribution of QoL index', fontsize=16, color='b')
plt.show()

In [11]:
from keras.src.legacy.preprocessing.image import ImageDataGenerator

# create datagenerator object for training and validation datasets
#rescale images using 1/255
eg_train_datagen = ImageDataGenerator(
            #we will do normalization for image pixel values as following
            rescale=1 / 255
            )

#use flow_from_dataframe method to load images from directory and labels from dataframe
eg_train_datagen_flow = eg_train_datagen.flow_from_dataframe(
    dataframe = eg_labels,
    directory='../outputs/tiles',
    x_col="tile", 
    y_col="qol_index",
    #we are doing regression, so we will assign class_mode to 'raw'
    class_mode="raw",
    #to convert all images to same pixel size, for neural networks, all images should have similar size
    target_size=(256,256),
    #we will load images batch by batch (every time 32 images will be loaded)
    batch_size=32,
    seed=42,
    )

In [12]:
#get one batch from our datagenerator and display images in it
features, target = next(eg_train_datagen_flow)

# display 16 images
fig = plt.figure(figsize=(12,12))
for i in range(32):
    fig.add_subplot(4, 8, i+1)
    plt.imshow(features[i])
    plt.title(f'{round(target[i], 2)}')
	# remove axes and place the images closer to one another for a more compact output
    plt.xticks([])
    plt.yticks([])
    plt.suptitle('Photos with QoL Index',  y=0.9,fontsize=16, color='b')
    plt.tight_layout()

## More concrete functions

In [ ]:
# TODO: Train, validation AND TEST
def load_dataset(subset):
    
    """
    Loads the subset (training/validation) of the data from path
    """
    
    labels = gpd.read_file('../outputs/matched/gauteng-qol-cluster-tiles.geojson')
    labels = labels[["tile", "qol_index"]]
    data = ImageDataGenerator(validation_split=0.2, rescale=1 / 255)
    data_flow = data.flow_from_dataframe(
        dataframe=labels,
        directory="../outputs/tiles",
        x_col='tile',
        y_col='qol_index',
        target_size=(256, 256),
        batch_size=32,
        class_mode='raw',
        subset = subset,
        seed=42)

    return data_flow

In [14]:

from keras.src.layers import Dropout, GlobalAveragePooling2D, Dense

def create_model(input_shape):
    
    """
    Defines the model
    """
    # Using ResNet50 architecture - freezing base model
    base_model = ResNet50V2(input_shape=input_shape, weights='imagenet', include_top=False)
    base_model.trainable = False

    # Create new model on top
    # Specify input shape
    inputs = Input(shape=(256, 256, 3))
    
    # New model is base model with training set to false
    x = base_model(inputs, training=False)
    # Add averaging layer to ensure fixed size vector
    x = GlobalAveragePooling2D()(x)
    # Add dropout layer to reduce overfitting
    x = Dropout(0.2)(x)
        
    #final layer, since we are doing regression we will add only one neuron (unit)
    outputs = Dense(1, activation='relu')(x)
    added_model = Model(inputs, outputs)

    print(added_model.summary())

    return base_model, added_model

In [15]:
#load training and testing (actually this is validation data set)
train = load_dataset("training")
validation = load_dataset("validation")

In [16]:
base_model, model = create_model(input_shape = (256, 256, 3))

In [ ]:
model.compile(
    optimizer=Adam(),
    loss=MeanSquaredError(),
)

In [ ]:
# checkpoint
filepath="../outputs/checkpoints/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [17]:
epochs = 20
print("Fitting the top layer of the model")
# TODO: What does this mean again?
history_a = model.fit(train, epochs=epochs, validation_data=validation, batch_size=10, callbacks=callbacks_list)

In [18]:
import pandas as pd

In [19]:
historya_df = pd.DataFrame(history_a.history)
historya_df.loc[0:, ['loss', 'val_loss']].plot()

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary(show_trainable=True)

model.compile(
    optimizer=Adam(1e-5),  # Low learning rate
    loss=MeanSquaredError(),
)

epochs = 100
print("Fitting the end-to-end model")
history_b = model.fit(train, epochs=epochs, validation_data=validation)

In [ ]:
historyb_df = pd.DataFrame(history_b.history)
historyb_df.loc[0:, ['loss', 'val_loss']].plot()